# 1. `공인맛집` 프로젝트의 목표

**데이터셋: https://github.com/seoul-opengov/opengov**

우리는 '맛집' 홍수의 시대에 살고 있습니다. 네이버 블로그 후기, 인스타그램 #을지로맛집, 망고플레이트, 지도 앱 평점, ... 여러 출처에서 맛집 정보를 찾을 때 가짜 리뷰를 분간해내는 일은 제1세계 문제 중에 하나죠.

그런데 2018년 새해, 디시인사이드 기타음식 갤러리에 **[필자가 블로그, sns거르고 맛집찾는법 알려준다](https://gall.dcinside.com/board/view/?id=food&no=1272073)**라는 글이 올라왔습니다. 관공서의 웹사이트에 가면 고위(2-5급) 공무원들의 업무추진비 내역이 공개되어 있는데, 업무추진비 내역을 통해 "고위 공무원이 손님을 대접하고 즐겨찾는, 정갈하고 조용한, 맛좋고 가격 좀 나가는 진짜 맛집"을 찾을 수 있다는 것입니다.

수많은 관공서 중에, "업무 추진비" 내역을 가장 깔끔하게 공개하고 있는 곳은 바로 **서울특별시청**입니다. 웹 스크레이핑 필요 없이 자료가 github에 올라와 있기도 하고, 내역이 매우 상세합니다.

**이 프로젝트는 서울특별시청들의 공무원들이 즐겨찾는 맛집을 분석하는 것을 목표로 합니다.**

# 2. 데이터 형태
현재 `서울시 행정정보 공개 GitHub`에 공개되어 있는 업무 추진비 사용 내역은, **2017년 1월부터 2020년 6월까지, 총 244,009건**입니다. Unique한 식당의 개수는 약 30,067개(추정)입니다.
* `nid`: 고유 번호
* `title`: 
* `url`:
* `dept_nm_lvl_1`, `dept_nm_lvl_2`, `dept_nm_lvl_3`, `dept_nm_lvl_4`, `dept_nm_lvl_5`: 부서명
* `exec_yr`: 사용 연도
* `exec_month`: 사용한 달
* `expense_budget`: 사용 금액
* `expense_execution`: 
* `category`: 구분
* `dept_nm_full`: 부서명
* `exec_dt`: 사용 일시
* `exec_loc`: 사용 장소
* `exec_purpose`: 사용 목적
* `target_nm`: 사용자 및 인원
* `payment_method`: 결제 방법
* `exec_amount`: 사용 금액

# 3. 분석 방향
1. 서울특별시청 소속 공무원들이 가장 많이 방문한 식당
2. 서울특별시청 소속 공무원들이 가장 많은 금액을 지출한 식당
3. 식당별 1인당 평균 금액 (가성비를 보기 위함)

# 4. 분석 과정

### * Dependencies

In [1]:
!pip install pandas
import pandas as pd
import glob

## 데이터 수집

In [2]:
SEOUL_OPENGOV_GITHUB_URL = "https://github.com/seoul-opengov/opengov/raw/master/"

start_yr, start_mo = 2017, 1
end_yr, end_mo = 2020, 6

def get_download_urls(begin_yr, begin_mo, end_yr, end_mo):
    """
    지정한 시작연월/끝연월 기간 동안의 업무추진비 내역 csv 파일들의 다운로드 URL 리스트를 반환합니다.
    
    Parameters
    ----------
    begin_yr: 시작년도
    begin_mo: 시작월
    end_yr: 종료년도
    end_mo: 종료월
    
    Yields
    ------
    list
    """
    urls = []
    for yr in range(begin_yr, end_yr+1):
        for mo in range(begin_mo, 13):
            if yr == end_yr and mo == end_mo + 1: break
            url = SEOUL_OPENGOV_GITHUB_URL + "expense_list" + str(yr) + "/" + str(yr) + str(mo).zfill(2) + "_expense_list.csv"
            urls.append(url)
            
    return urls

def save_data(urls):
    """
    URL 리스트의 CSV 파일을 다운로드해 저장합니다.
    """
    for url in urls:
        filename = url.split('/')[-1]
        df = pd.read_csv(url)
        df.to_csv("./data/"+filename)
        
urls = get_download_urls(2017, 1, 2020, 6)
save_data(urls)

## 데이터 전처리


### 1. Merge

In [3]:
filenames = sorted(glob.glob('./data/*.csv'))

merged = pd.concat([pd.read_csv(f) for f in filenames])

### 2. 식당명

In [4]:
# 1. 공백 제거 91887 -> 91660
merged['exec_loc'] = merged['exec_loc'].str.strip()
# 2. (중구 ~~) 91660 -> 50158
merged['exec_loc'] = merged['exec_loc'].str.replace(r'\( *.*구.+ *\)','')
# 3. (주) 50158 -> 41647
merged['exec_loc'] = merged['exec_loc'].str.replace(r'(\(주\)|（주）|㈜|（주\))','')
# 4. 기타 41647 -> 41598
merged['exec_loc'] = merged['exec_loc'].str.replace(r'("|-)','')
# 5. 서울 ~구 41598 -> 39595
merged['exec_loc'] = merged['exec_loc'].str.replace(r'\(*서울.+구.+\)*','')
# 6. 구 동|길 38202 -> 36781
merged['exec_loc'] = merged['exec_loc'].str.replace(r' .+구.+[동|길] *\d*','')
# 7. 길동시도군36781 -> 32871
merged['exec_loc'] = merged['exec_loc'].str.replace(r'\((.+길|.+동|.+시|.+도|.+군|.+로) *\d*.*\)*','')
# 8. 띄어쓰기 제거 32871 -> 29935
merged['exec_loc'] = merged['exec_loc'].str.replace(' ','')
# 9. 특수
merged['exec_loc'] = merged['exec_loc'].str.replace('창고43시청점','창고43')

### 3. price per person
정확하지 않을 수 있음.

In [7]:
import re
def target_nm_to_num(row):
    try:
        match = re.findall('(\d+)[명인]', row['target_nm'])
    except Exception as e:
        return None
    if len(match) != 1:
        return None
    else:
        num = int(match[0])
        if '외' in row['target_nm']:
            num +=1
        return num

merged['num_people'] = merged.apply(target_nm_to_num, axis=1)
merged['price_per_person'] = merged['exec_amount'] / merged['num_people']

## 방문 횟수 랭킹

In [8]:
visit_ranking = merged['exec_loc'].value_counts()
visit_ranking.head(50)

               6628
서울시청매점         2637
참숯골            2496
우도일식           2276
구이구이           2011
창고43           1906
곰국시집           1765
삼우정            1504
동해일식           1491
엠엠피아이          1452
바닷가작은부엌        1445
무교소호정          1321
오리마당           1232
뚜리삼            1110
더테이블한정식        1088
이마트양재점         1069
월향              976
만복림             872
스타벅스            839
월매네남원추어탕        819
복성각             817
파리바게뜨           814
라칸티나            794
무교동낙지           737
동원참치            727
해피머니아이엔씨        709
어미가             705
열빈              705
영덕회식당           702
남포면옥            687
목포세발낙지          679
동해수산            674
이마트             674
잼배옥             664
대복              655
훈민정             648
서라벌             640
금강산             636
라그릴리아광화문점       614
서울삼계탕           614
참복집             608
라그릴리아           597
바닷가작은부엌덕수궁점     594
배수사             591
여수바다장어          581
한국문화진흥          569
창고43무교점         568
원주추어탕           558
지수마트            549
남산복집            537


## 사용 금액 랭킹

In [9]:
amount_ranking = merged.groupby('exec_loc')['exec_amount'].agg(sum).astype('int64').nlargest(50)
amount_ranking

exec_loc
               1104888601
서울시청매점          702615380
참숯골             322704570
우도일식            292609908
엠엠피아이           277373321
삼우정             259666650
동해일식            234093360
곰국시집            219441550
바닷가작은부엌         218965400
창고43            208818345
구이구이            205100040
해피머니아이엔씨        178690261
뚜리삼             159891630
무교소호정           153308400
만복림             153240870
한국문화진흥          147414860
라칸티나            138477800
더테이블한정식         138332420
오리마당            137656951
열빈              133124540
이마트양재점          125960420
지수마트            124700010
서울시청연금매점        118000460
월향              116746430
복성각             114547455
동해수산            107428420
서울시청신매점         106864980
시청매점            106162180
배수사             102022000
라그릴리아광화문점        99336067
동원참치             98350550
이마트              96160540
더미               91566800
대복               90916600
라그릴리아            88906330
삼원일식             87608140
바닷가작은부엌덕수궁점      87386970
더미시청점            87368368
본청직

## 방문 횟수 랭킹 + 1인당 평균 가격

In [10]:
mean_price_per_person = merged.groupby('exec_loc')['price_per_person'].agg("mean")

In [11]:
visit_mean_price_ranking = pd.concat((visit_ranking, mean_price_per_person), axis=1)
visit_mean_price_ranking.head(50)

,exec_loc,price_per_person
,6628,16112.623916
서울시청매점,2637,11341.031457
참숯골,2496,16681.995816
우도일식,2276,19028.197593
구이구이,2011,16606.651481
창고43,1906,15391.041618
곰국시집,1765,18332.453406
삼우정,1504,17647.408883
동해일식,1491,21724.021304
엠엠피아이,1452,8012.595103
